<a href="https://colab.research.google.com/github/nonyeezeh/Research-Project-Code/blob/main/LBN_Dense_BIC_15_Nodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 40.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from pgmpy.estimators import HillClimbSearch, BicScore, MaximumLikelihoodEstimator
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import BayesianEstimator
from sklearn.model_selection import train_test_split
from scipy.stats import entropy
from tabulate import tabulate

# Bayesian Network Data Generation 500, ..., 20000 Samples (dense)

In [ ]:
# Function to generate CPDs for the new structure with 15 nodes
def generate_cpds():
    # Generate random probabilities for IR
    ir_probs = np.random.rand(3)
    ir_probs /= ir_probs.sum()

    # Generate random probabilities for IRT given IR
    irt_given_ir_probs = np.random.rand(3, 3)
    irt_given_ir_probs /= irt_given_ir_probs.sum(axis=0, keepdims=True)

    # Generate random probabilities for EI given IR and IRT
    ei_given_ir_irt_probs = np.random.rand(3, 3, 3)
    ei_given_ir_irt_probs /= ei_given_ir_irt_probs.sum(axis=0, keepdims=True)

    # Generate random probabilities for GEO
    geo_probs = np.random.rand(3)
    geo_probs /= geo_probs.sum()

    # Generate random probabilities for UE given GEO
    ue_given_geo_probs = np.random.rand(3, 3)
    ue_given_geo_probs /= ue_given_geo_probs.sum(axis=0, keepdims=True)

    # Generate random probabilities for MS given EI and GEO
    ms_given_ei_geo_probs = np.random.rand(3, 3, 3)
    ms_given_ei_geo_probs /= ms_given_ei_geo_probs.sum(axis=0, keepdims=True)

    # Generate random probabilities for IRP given IR and IRT
    irp_given_ir_irt_probs = np.random.rand(3, 3, 3)
    irp_given_ir_irt_probs /= irp_given_ir_irt_probs.sum(axis=0, keepdims=True)

    # Generate random probabilities for INF given MS and IRP
    inf_given_ms_irp_probs = np.random.rand(3, 3, 3)
    inf_given_ms_irp_probs /= inf_given_ms_irp_probs.sum(axis=0, keepdims=True)

    # Generate random probabilities for MP given IR and UE
    mp_given_ir_ue_probs = np.random.rand(3, 3, 3)
    mp_given_ir_ue_probs /= mp_given_ir_ue_probs.sum(axis=0, keepdims=True)

    # Generate random probabilities for GS given INF and MS
    gs_given_inf_ms_probs = np.random.rand(3, 3, 3)
    gs_given_inf_ms_probs /= gs_given_inf_ms_probs.sum(axis=0, keepdims=True)

    # Generate random probabilities for CC given GS and MP
    cc_given_gs_mp_probs = np.random.rand(3, 3, 3)
    cc_given_gs_mp_probs /= cc_given_gs_mp_probs.sum(axis=0, keepdims=True)

    # Generate random probabilities for FI given GS and CC
    fi_given_gs_cc_probs = np.random.rand(3, 3, 3)
    fi_given_gs_cc_probs /= fi_given_gs_cc_probs.sum(axis=0, keepdims=True)

    # Generate random probabilities for DL given FI and UE
    dl_given_fi_ue_probs = np.random.rand(3, 3, 3)
    dl_given_fi_ue_probs /= dl_given_fi_ue_probs.sum(axis=0, keepdims=True)

    # Generate random probabilities for ED given CC and MP
    ed_given_cc_mp_probs = np.random.rand(3, 3, 3)
    ed_given_cc_mp_probs /= ed_given_cc_mp_probs.sum(axis=0, keepdims=True)

    # Generate random probabilities for SP given IR, EI, UE, INF, CC, FI, DL, and ED
    sp_probs = np.random.rand(3, 3, 3, 3, 3, 3, 3, 3, 3)
    sp_probs /= sp_probs.sum(axis=0, keepdims=True)

    return (ir_probs, irt_given_ir_probs, ei_given_ir_irt_probs, geo_probs, ue_given_geo_probs,
            ms_given_ei_geo_probs, irp_given_ir_irt_probs, inf_given_ms_irp_probs, mp_given_ir_ue_probs,
            gs_given_inf_ms_probs, cc_given_gs_mp_probs, fi_given_gs_cc_probs, dl_given_fi_ue_probs,
            ed_given_cc_mp_probs, sp_probs)

# Function to generate and save samples with the new nodes
def generate_and_save_samples(ir_probs, irt_probs, ei_probs, geo_probs, ue_probs, ms_probs, irp_probs, inf_probs, mp_probs, gs_probs, cc_probs, fi_probs, dl_probs, ed_probs, sp_probs, sample_size, filename):
    output_data = []

    # Generate `sample_size` random samples
    for _ in range(sample_size):
        # Sample IR
        ir_state_idx = np.random.choice(3, p=ir_probs)
        ir_state = ['low', 'medium', 'high'][ir_state_idx]
        ir_prob = ir_probs[ir_state_idx]

        # Sample IRT given IR
        irt_probs_given_ir = irt_probs[:, ir_state_idx]
        irt_state_idx = np.random.choice(3, p=irt_probs_given_ir)
        irt_state = ['decreasing', 'steady', 'increasing'][irt_state_idx]
        irt_prob = irt_probs_given_ir[irt_state_idx]

        # Sample EI given IR and IRT
        ei_probs_given_ir_irt = ei_probs[:, ir_state_idx, irt_state_idx]
        ei_state_idx = np.random.choice(3, p=ei_probs_given_ir_irt)
        ei_state = ['poor', 'average', 'good'][ei_state_idx]
        ei_prob = ei_probs_given_ir_irt[ei_state_idx]

        # Sample GEO
        geo_state_idx = np.random.choice(3, p=geo_probs)
        geo_state = ['recession', 'stable', 'growth'][geo_state_idx]
        geo_prob = geo_probs[geo_state_idx]

        # Sample UE given GEO
        ue_probs_given_geo = ue_probs[:, geo_state_idx]
        ue_state_idx = np.random.choice(3, p=ue_probs_given_geo)
        ue_state = ['high', 'medium', 'low'][ue_state_idx]
        ue_prob = ue_probs_given_geo[ue_state_idx]

        # Sample MS given EI and GEO
        ms_probs_given_ei_geo = ms_probs[:, ei_state_idx, geo_state_idx]
        ms_state_idx = np.random.choice(3, p=ms_probs_given_ei_geo)
        ms_state = ['negative', 'neutral', 'positive'][ms_state_idx]
        ms_prob = ms_probs_given_ei_geo[ms_state_idx]

        # Sample IRP given IR and IRT
        irp_probs_given_ir_irt = irp_probs[:, ir_state_idx, irt_state_idx]
        irp_state_idx = np.random.choice(3, p=irp_probs_given_ir_irt)
        irp_state = ['loose', 'neutral', 'tight'][irp_state_idx]
        irp_prob = irp_probs_given_ir_irt[irp_state_idx]

        # Sample INF given MS and IRP
        inf_probs_given_ms_irp = inf_probs[:, ms_state_idx, irp_state_idx]
        inf_state_idx = np.random.choice(3, p=inf_probs_given_ms_irp)
        inf_state = ['deflation', 'stable', 'inflation'][inf_state_idx]
        inf_prob = inf_probs_given_ms_irp[inf_state_idx]

        # Sample MP given IR and UE
        mp_probs_given_ir_ue = mp_probs[:, ir_state_idx, ue_state_idx]
        mp_state_idx = np.random.choice(3, p=mp_probs_given_ir_ue)
        mp_state = ['expansionary', 'neutral', 'contractionary'][mp_state_idx]
        mp_prob = mp_probs_given_ir_ue[mp_state_idx]

        # Sample GS given INF and MS
        gs_probs_given_inf_ms = gs_probs[:, inf_state_idx, ms_state_idx]
        gs_state_idx = np.random.choice(3, p=gs_probs_given_inf_ms)
        gs_state = ['low', 'moderate', 'high'][gs_state_idx]
        gs_prob = gs_probs_given_inf_ms[gs_state_idx]

        # Sample CC given GS and MP
        cc_probs_given_gs_mp = cc_probs[:, gs_state_idx, mp_state_idx]
        cc_state_idx = np.random.choice(3, p=cc_probs_given_gs_mp)
        cc_state = ['low', 'moderate', 'high'][cc_state_idx]
        cc_prob = cc_probs_given_gs_mp[cc_state_idx]

        # Sample FI given GS and CC
        fi_probs_given_gs_cc = fi_probs[:, gs_state_idx, cc_state_idx]
        fi_state_idx = np.random.choice(3, p=fi_probs_given_gs_cc)
        fi_state = ['low', 'medium', 'high'][fi_state_idx]
        fi_prob = fi_probs_given_gs_cc[fi_state_idx]

        # Sample DL given FI and UE
        dl_probs_given_fi_ue = dl_probs[:, fi_state_idx, ue_state_idx]
        dl_state_idx = np.random.choice(3, p=dl_probs_given_fi_ue)
        dl_state = ['low', 'medium', 'high'][dl_state_idx]
        dl_prob = dl_probs_given_fi_ue[dl_state_idx]

                # Sample ED given CC and MP
        ed_probs_given_cc_mp = ed_probs[:, cc_state_idx, mp_state_idx]
        ed_state_idx = np.random.choice(3, p=ed_probs_given_cc_mp)
        ed_state = ['low', 'moderate', 'high'][ed_state_idx]
        ed_prob = ed_probs_given_cc_mp[ed_state_idx]

        # Sample SP given IR, EI, UE, INF, CC, FI, DL, and ED
        sp_probs_given_ir_ei_ue_inf_cc_fi_dl_ed = sp_probs[:, ir_state_idx, ei_state_idx, ue_state_idx, inf_state_idx, cc_state_idx, fi_state_idx, dl_state_idx, ed_state_idx]
        sp_state_idx = np.random.choice(3, p=sp_probs_given_ir_ei_ue_inf_cc_fi_dl_ed)
        sp_state = ['decrease', 'stable', 'increase'][sp_state_idx]
        sp_prob = sp_probs_given_ir_ei_ue_inf_cc_fi_dl_ed[sp_state_idx]

        # Append sample data to output list including probabilities for all nodes
        output_data.append({
            'IR_State': ir_state,
            'IR_Prob': f'{ir_prob:.4f}',
            'IRT_State': irt_state,
            'IRT_Prob': f'{irt_prob:.4f}',
            'EI_State': ei_state,
            'EI_Prob': f'{ei_prob:.4f}',
            'GEO_State': geo_state,
            'GEO_Prob': f'{geo_prob:.4f}',
            'UE_State': ue_state,
            'UE_Prob': f'{ue_prob:.4f}',
            'MS_State': ms_state,
            'MS_Prob': f'{ms_prob:.4f}',
            'IRP_State': irp_state,
            'IRP_Prob': f'{irp_prob:.4f}',
            'INF_State': inf_state,
            'INF_Prob': f'{inf_prob:.4f}',
            'MP_State': mp_state,
            'MP_Prob': f'{mp_prob:.4f}',
            'GS_State': gs_state,
            'GS_Prob': f'{gs_prob:.4f}',
            'CC_State': cc_state,
            'CC_Prob': f'{cc_prob:.4f}',
            'FI_State': fi_state,
            'FI_Prob': f'{fi_prob:.4f}',
            'DL_State': dl_state,
            'DL_Prob': f'{dl_prob:.4f}',
            'ED_State': ed_state,
            'ED_Prob': f'{ed_prob:.4f}',
            'SP_Probabilities (decrease, stable, increase)': ', '.join([f'{prob:.4f}' for prob in sp_probs_given_ir_ei_ue_inf_cc_fi_dl_ed]),
            'Chosen_SP_State': sp_state,
            'Chosen_SP_Probability': f'{sp_prob:.4f}'
        })

    # Create a DataFrame from the output data
    output_df = pd.DataFrame(output_data)

    # Save the output DataFrame to a CSV file
    output_df.to_csv(filename, index=False)

    # Print the first few rows for visual confirmation
    print(f"\nSample size: {sample_size} - First few rows of generated samples:\n")
    print(tabulate(output_df.head(), headers='keys', tablefmt='grid'))

# Generate and save samples for sample sizes
sample_sizes = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 19000, 20000]

for size in sample_sizes:
    ir_probs, irt_probs, ei_probs, geo_probs, ue_probs, ms_probs, irp_probs, inf_probs, mp_probs, gs_probs, cc_probs, fi_probs, dl_probs, ed_probs, sp_probs = generate_cpds()
    generate_and_save_samples(ir_probs, irt_probs, ei_probs, geo_probs, ue_probs, ms_probs, irp_probs, inf_probs, mp_probs, gs_probs, cc_probs, fi_probs, dl_probs, ed_probs, sp_probs, size, f'combined_probabilities_{size}.csv')

# Notify the user that the process is done
print("\nGeneration and saving of individual samples complete for all sample sizes!")


Sample size: 50 - First few rows of generated samples:

+----+------------+-----------+-------------+------------+------------+-----------+-------------+------------+------------+-----------+------------+-----------+-------------+------------+-------------+------------+----------------+-----------+------------+-----------+------------+-----------+------------+-----------+------------+-----------+------------+-----------+-------------------------------------------------+-------------------+-------------------------+
|    | IR_State   |   IR_Prob | IRT_State   |   IRT_Prob | EI_State   |   EI_Prob | GEO_State   |   GEO_Prob | UE_State   |   UE_Prob | MS_State   |   MS_Prob | IRP_State   |   IRP_Prob | INF_State   |   INF_Prob | MP_State       |   MP_Prob | GS_State   |   GS_Prob | CC_State   |   CC_Prob | FI_State   |   FI_Prob | DL_State   |   DL_Prob | ED_State   |   ED_Prob | SP_Probabilities (decrease, stable, increase)   | Chosen_SP_State   |   Chosen_SP_Probability |
+====+=======

# LBN Dense BIC & Entropy

In [ ]:
# Sample sizes to loop through
sample_sizes = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 19000, 20000]

# Initialize list to store K-L divergence and standard deviation values for each sample size
results = []

# Small smoothing value to avoid zero probabilities
epsilon = 1e-10

# Loop through each sample size
for sample_size in sample_sizes:
    print(f"\nProcessing sample size: {sample_size}")

    # Load the dataset for the current sample size
    data_file = f'combined_probabilities_{sample_size}.csv'
    df = pd.read_csv(data_file)

    # Manually encode categorical variables for IR, IRT, EI, GEO, UE, MS, IRP, INF, MP, GS, CC, FI, DL, ED, and SP
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    irt_map = {'decreasing': 0, 'steady': 1, 'increasing': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}
    geo_map = {'recession': 0, 'stable': 1, 'growth': 2}
    ue_map = {'high': 0, 'medium': 1, 'low': 2}
    ms_map = {'negative': 0, 'neutral': 1, 'positive': 2}
    irp_map = {'loose': 0, 'neutral': 1, 'tight': 2}
    inf_map = {'deflation': 0, 'stable': 1, 'inflation': 2}
    mp_map = {'expansionary': 0, 'neutral': 1, 'contractionary': 2}
    gs_map = {'low': 0, 'moderate': 1, 'high': 2}
    cc_map = {'low': 0, 'moderate': 1, 'high': 2}
    fi_map = {'low': 0, 'medium': 1, 'high': 2}
    dl_map = {'low': 0, 'medium': 1, 'high': 2}
    ed_map = {'low': 0, 'moderate': 1, 'high': 2}
    sp_map = {'decrease': 0, 'stable': 1, 'increase': 2}

    df['IR_encoded'] = df['IR_State'].map(ir_map)
    df['IRT_encoded'] = df['IRT_State'].map(irt_map)
    df['EI_encoded'] = df['EI_State'].map(ei_map)
    df['GEO_encoded'] = df['GEO_State'].map(geo_map)
    df['UE_encoded'] = df['UE_State'].map(ue_map)
    df['MS_encoded'] = df['MS_State'].map(ms_map)
    df['IRP_encoded'] = df['IRP_State'].map(irp_map)
    df['INF_encoded'] = df['INF_State'].map(inf_map)
    df['MP_encoded'] = df['MP_State'].map(mp_map)
    df['GS_encoded'] = df['GS_State'].map(gs_map)
    df['CC_encoded'] = df['CC_State'].map(cc_map)
    df['FI_encoded'] = df['FI_State'].map(fi_map)
    df['DL_encoded'] = df['DL_State'].map(dl_map)
    df['ED_encoded'] = df['ED_State'].map(ed_map)
    df['SP_encoded'] = df['Chosen_SP_State'].map(sp_map)

    # Split the data into training, validation, and test sets
    X = df[['IR_encoded', 'IRT_encoded', 'EI_encoded', 'GEO_encoded', 'UE_encoded', 'MS_encoded', 'IRP_encoded', 'INF_encoded', 'MP_encoded', 'GS_encoded', 'CC_encoded', 'FI_encoded', 'DL_encoded', 'ED_encoded']]
    y = df['SP_encoded']

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, shuffle=True)

    # Concatenate X and y to form the training set for learning the BN structure
    df_train = pd.concat([X_train, y_train], axis=1)

    # Enforce priors (fixed edges)
    priors = [
        ('IR_encoded', 'SP_encoded'),
        ('IRT_encoded', 'IR_encoded'),
        ('EI_encoded', 'SP_encoded'),
        ('UE_encoded', 'SP_encoded'),
        ('GEO_encoded', 'UE_encoded'),
        ('MS_encoded', 'EI_encoded'),
        ('IRP_encoded', 'IR_encoded'),
        ('INF_encoded', 'MS_encoded'),
        ('INF_encoded', 'IRP_encoded'),
        ('MP_encoded', 'IR_encoded'),
        ('MP_encoded', 'UE_encoded'),
        ('GS_encoded', 'INF_encoded'),
        ('GS_encoded', 'MS_encoded'),
        ('CC_encoded', 'GS_encoded'),
        ('CC_encoded', 'MP_encoded'),
        ('FI_encoded', 'GS_encoded'),
        ('FI_encoded', 'CC_encoded'),
        ('DL_encoded', 'FI_encoded'),
        ('DL_encoded', 'UE_encoded'),
        ('ED_encoded', 'CC_encoded'),
        ('ED_encoded', 'MP_encoded')
    ]

    # Perform Hill Climb Search with priors
    hc = HillClimbSearch(df_train)
    best_dag = hc.estimate(scoring_method=BicScore(df_train), fixed_edges=priors)

    # Initialize BayesianNetwork and print edges
    best_model = BayesianNetwork(best_dag.edges())
    print("Learned structure (edges):", best_model.edges())

    if len(best_model.edges()) == 0:
        print("No edges learned. Skipping to next sample size.")
        continue

    # Try fitting parameters using BayesianEstimator
    try:
        best_model.fit(df_train, estimator=BayesianEstimator, prior_type="BDeu", equivalent_sample_size=5)
        print("Model fitted successfully.")
    except Exception as e:
        print("Error during fitting:", str(e))
        continue

    # Check model validity
    if not best_model.check_model():
        print("Model check failed.")
        continue

    # Predict on test data and calculate K-L divergence
    try:
        inference = best_model.predict(X_test)
        predicted_probabilities = inference['SP_encoded']

        ground_truth_probabilities = y_test.value_counts(normalize=True).sort_index()
        predicted_probabilities = predicted_probabilities.value_counts(normalize=True).sort_index()

        # Reindex both distributions and add smoothing
        all_categories = sorted(set(ground_truth_probabilities.index).union(set(predicted_probabilities.index)))
        ground_truth_probabilities = ground_truth_probabilities.reindex(all_categories, fill_value=epsilon)
        predicted_probabilities = predicted_probabilities.reindex(all_categories, fill_value=epsilon)

        kl_divergence = entropy(pk=ground_truth_probabilities, qk=predicted_probabilities)
        std_dev = np.std(predicted_probabilities - ground_truth_probabilities)

        results.append({
            'Sample_Size': sample_size,
            'K-L_Divergence': kl_divergence,
            'Standard_Deviation': std_dev
        })

        print(f"K-L Divergence: {kl_divergence:.4f}")
        print(f"Standard Deviation: {std_dev:.4f}")

    except Exception as e:
        print("Error during prediction:", str(e))

# Save the K-L divergence results
results_df = pd.DataFrame(results)
results_df.to_csv('kl_std_bic_with_new_nodes.csv', index=False)

print("\nAll results have been saved.")


Processing sample size: 50


  0%|          | 0/1000000 [00:00<?, ?it/s]

Learned structure (edges): [('IR_encoded', 'SP_encoded'), ('IRT_encoded', 'IR_encoded'), ('EI_encoded', 'SP_encoded'), ('GEO_encoded', 'UE_encoded'), ('UE_encoded', 'SP_encoded'), ('MS_encoded', 'EI_encoded'), ('IRP_encoded', 'IR_encoded'), ('INF_encoded', 'MS_encoded'), ('INF_encoded', 'IRP_encoded'), ('MP_encoded', 'UE_encoded'), ('MP_encoded', 'IR_encoded'), ('GS_encoded', 'INF_encoded'), ('GS_encoded', 'MS_encoded'), ('CC_encoded', 'MP_encoded'), ('CC_encoded', 'GS_encoded'), ('FI_encoded', 'CC_encoded'), ('FI_encoded', 'GS_encoded'), ('DL_encoded', 'FI_encoded'), ('DL_encoded', 'UE_encoded'), ('ED_encoded', 'MP_encoded'), ('ED_encoded', 'CC_encoded')]
Model fitted successfully.


  0%|          | 0/8 [00:00<?, ?it/s]

K-L Divergence: 0.6931
Standard Deviation: 0.3062

Processing sample size: 100


...

K-L Divergence: 0.1546
Standard Deviation: 0.1559

Processing sample size: 20000


  0%|          | 0/1000000 [00:00<?, ?it/s]

Learned structure (edges): [('IR_encoded', 'SP_encoded'), ('IR_encoded', 'UE_encoded'), ('IR_encoded', 'EI_encoded'), ('UE_encoded', 'SP_encoded'), ('EI_encoded', 'SP_encoded'), ('EI_encoded', 'GEO_encoded'), ('IRT_encoded', 'IR_encoded'), ('IRT_encoded', 'EI_encoded'), ('GEO_encoded', 'UE_encoded'), ('MS_encoded', 'EI_encoded'), ('MS_encoded', 'GEO_encoded'), ('MS_encoded', 'IRP_encoded'), ('IRP_encoded', 'IR_encoded'), ('IRP_encoded', 'IRT_encoded'), ('INF_encoded', 'MS_encoded'), ('INF_encoded', 'IRP_encoded'), ('MP_encoded', 'UE_encoded'), ('MP_encoded', 'IR_encoded'), ('MP_encoded', 'IRT_encoded'), ('GS_encoded', 'INF_encoded'), ('GS_encoded', 'MS_encoded'), ('GS_encoded', 'MP_encoded'), ('CC_encoded', 'MP_encoded'), ('CC_encoded', 'GS_encoded'), ('FI_encoded', 'CC_encoded'), ('FI_encoded', 'GS_encoded'), ('FI_encoded', 'ED_encoded'), ('ED_encoded', 'MP_encoded'), ('ED_encoded', 'CC_encoded'), ('DL_encoded', 'FI_encoded'), ('DL_encoded', 'UE_encoded')]
Model fitted successfully.


  0%|          | 0/2969 [00:00<?, ?it/s]

K-L Divergence: 0.0714
Standard Deviation: 0.1128

All results have been saved.
